# Simulated Annealing

Meter antes el problema, luego la generalizacion, o primero la generalización, pasar las funciones a la funcion y luego el problema en especifico con las funciones especificas.

#### By Jorge Almonacid


Simulated Annealing is an iterative algorithm that explores the solution space by occasionally accepting worse solutions with a probability that decreases over time, enabling the algorithm to escape local optima and eventually settle into a global optimum (or a good approximation).

In [47]:
# Library imports

import numpy as np
import pandas as pd
import math

In [48]:
def list_to_str(lista:list):
    """ 
    This function is for converting a list into a string format, so the dataframe can save it
    as a single value. 
    E.g. lista = [1,0,1,1] -> t = 1011
    
    Args: 
        Lista (list): list to transform into string
        
    Return: 
        t (str): String associated to the list
    """
    
    return "".join([str(elem) for elem in lista])

def simulated_annealing(lista1: list, temp: float, pesos: list, max_cost: float, coste: list, cool_rate: float, df: pd.DataFrame, evaluation, neighbourhood, cooling, valid) -> list:
    """ 
    This function solves a simulated annealing problem given the starting point list, initial temperature, weighs associated to the problem,
    maximum cost we want to not surpass, costs associated to the problem , cooling rate associated to the temperature and a dataframe
    to save the outputs later.
    
    Args:
        lista1 (list): Starting point for the problem. In our case a 0 filled n length matrix
        temp (float): Initial temperature 
        pesos (list): Weighs associated to the problem
        max_cost (float): Cost that we must not surpass
        coste (list): Costs associated to the problem
        cool_rate (float): Temperature decrease rate
        df (pd.DataFrame): Dataframe we will modify to the save it later in the code
        evaluation (function): This function is used to calculate de value of each node given a certain weighs
        neighbourhood (function): This function is used to calculate the neighbourhood nodes to the one given
        cooling (function): This function is used to calculate the new temperature after cooling
        valid (function): Checks whether the max cost is exceeded or not
        
    Returns:
        best (list): List with the maximum associated value we have found that does not surpass the maximum cost.
        best_eval (float): Value associated to the best list
        (float): Cost associated to the best list.
        df (pd.DataFrame): Dataframe with the outputs from the problem each 100 iterations ("Iteración","Lista", "Valor", "Peso", "Temperatura", "Energy")
    """
    # Firstly we initialize the solution and best lists, to keep track of them and define a function with the associated weighs for the problem
    solution = lista1.copy()
    eval = evaluation(pesos)
    best = lista1.copy()
    # Evaluate the best list
    best_eval = eval(best)
    ev1 = eval(solution)    

    i=0
    # While loop for evaluating until the temperature drops below 1
    while temp>1:
        # Generate random point between 0 and 1 = (exp(-(ev1-ev2)/temp)) to compare with the energy, when ev1 <= ev2
        rn = np.random.uniform(0,1)
        # Generate a neighbour to the current solution list
        lista2 = neighbourhood(solution)
        # Reduce the temperature
        temp = cooling(temp, cool_rate)
        # Evaluate both lists, current solution and possible solution
        ev2 = eval(lista2)

        # Check if the new list is exceeding the maximum cost to set its value to 0
        if not valid(max_cost, coste, lista2):
            ev2 = 0
        
        # Define the Δ between both evaluations and calculate the energy
        dif = ev2 - ev1
        energy = math.exp(dif/temp)
        
        # If the random value is smaller than the energy, or if ev2 > ev1, we set the solution as the new list
        if (dif > 0) or (energy > rn):
            solution = lista2.copy()
            ev1 = eval(solution)
        
        # If the new list has a greater evaluation we also modify the best solution
        if ev2 > best_eval:
            best = solution.copy()
            best_eval = eval(best)
        
        # Add 1 for dataframe iteration count
        i+=1
        
        if not valid(max_cost, coste, solution):
            ev1 = 0
        if i% 100 == 0:
            df2 = pd.DataFrame(np.array([[i,list_to_str(solution),ev1,np.dot(coste, solution),temp, energy]]), columns=["Iteración","Lista", "Valor", "Peso", "Temperatura", "Energy"])       
            df = pd.concat([df,df2])
    
    return best, best_eval, np.dot(coste, best), df


In [49]:
# Function Definition

def neighbourhood(lista1: list) -> list:
    """
    This function creates a list from the adjacent neighbour space to the list passed.
    E.g. lista1 = [0,0] -> lista =  [0,1] || [1,0]
    
    Args:
        lista1 (list): List you want to obtain the neighbour from
        
    Returns: 
        lista (list): Neighbour list   
    """
    lista = lista1.copy()
    i = np.random.randint(0, len(lista))
    lista[i] = 1 - lista[i]
    return lista

def evaluation(weighs:list) -> float:
    def compare(list1: list) -> list:
        """ 
        This function evaluates a list associated to its positional weighs. Both lists must contain numbers.
        E.g. weighs = [1,1,2,0], list1 = [0,1,1,0] -> [1*0 + 1*1 + 2*1 + 0*0] = 3
        
        Args:
            list1 (list): List you want to calculate the weighted value from
            weighs (list): Weighs associated to each list position
            
        Returns: 
            compare (function): Function with the associated weighs
            (float): Total added value from the list   
        """
        return np.dot(list1,weighs)
    return compare
    
def valid(max_coste:float, coste:list, lista:list) -> bool:
    """ 
    This function checks if the cost associated to a list exceeds or not the maximum cost.
    E.g. max_cost = 10, cost = [5,6,1], lista = [1,1,0]: 10 < (5*1 + 6*1)=11 -> False   
    
    Args: 
        max_coste (float): Max cost you can't surpass
        coste (list): Weighs associated to the list
        lista (list):  List you want to check if it's valid
        
    Returns:
        (bool): True if the value is under the maximum cost, False otherwise
    """
    coste_total = np.dot(coste, lista)
    if max_coste < coste_total:
        return False
    return True

def cooling(temp: float, cool_rate: float) -> float:
    """ 
    Function to reduce the temperature by some certain cooling rate.
    E.g. temp = 10, cool_rate = 0.9 -> 10*0.9 = 9
    
    Args:
        temp (float): Temperature value you want to decrease
        cool_rate (float): Temperature decrease rate
        
    Returns:
        (float): New temperature after cooling down
    """
    return cool_rate * temp

In [50]:
# Define the problem
# Weighs and costs associated to the problem
pesos = [12,10,20,15,18]
coste = [4,6,5,3,7]
n = len(coste)

sols = [0]*n

# Maximum cost, temperature and cooling rate
max_cost = 15
temp = 100
cool_rate = 0.999

# DataFrame initialization
df = pd.DataFrame(np.array([[0, list_to_str(sols), evaluation(pesos)(sols), 0, temp, 10]]), columns= ["Iteración","Lista", "Valor", "Peso", "Temperatura","Energy"])

# Problem solving
sol = simulated_annealing(sols, temp, pesos, max_cost, coste, cool_rate,df,evaluation, neighbourhood, cooling, valid)
print(sol[:3])

([0, 0, 1, 1, 1], np.int64(53), np.int64(15))


In [51]:
# Save solutions into a csv
df3 = sol[3]
df3.to_csv("simmulated_annealing_output.csv")
print(df3)

  Iteración  Lista Valor Peso         Temperatura                  Energy
0         0  00000     0    0                 100                      10
0       100  10110    47   12   90.47921471137093      0.8757915738200691
0       200  10010    27    7   81.86488294786366      0.8850133949040617
0       300  10000    12    4   74.07070321561004      1.1758689102684714
0       400  01100    30   11   67.01859060067416      1.1609194600859944
0       500  11000    22   10   60.63789448611864      1.4373619166220346
0       600  10001    30   11  54.864690748549855       1.727715066739056
0       700  10011    45   14   49.64114134311003      0.7392133363634625
0       800  11000    22   10  44.914914861007645       1.306264882609044
0       900  11010    37   13   40.63866225452056      0.4023378821421464
0      1000  01011     0   16   36.76954247709648     0.46696553700290455
0      1100  00011    33   10   33.26879328624084      0.6971892496870928
0      1200  00111    53   15  30.1013